In [1]:
#librerias estandar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [ ]:
data = pd.read_csv(r"C:\Users\USER\Desktop\intro_ML\train.csv", index_col="ID")
data.drop(columns=["F_TIENEINTERNET.1"], inplace=True)  #se elimina de una vez
data.head()

,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
ID,,,,,,,,,,,,,,,,,,,
904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from catboost import CatBoostClassifier

Después de explorar con los modelos vistos en clase, busqué otros y se me recomendó uno llamado catboost. Según documentación este modelo hace su propio preprocesamiento de los datos y por lo tanto no se debe preprocesar por aparte de una manera muy robusta. Probándolo observé que sí daba mejor este modelo, así que, como no conozco bien cómo funciona este modelo, escribí una rutina que iterara con distintos valores de hiperparámetros y se quedara con los mejores (esta rutina esta en el archivo 09). Estos hiperparámetros son los que se encuentran en este archivo

In [4]:
#modelo con mejores parametros

y = data["RENDIMIENTO_GLOBAL"]                 # variable a predecir
X = data.drop(columns=["RENDIMIENTO_GLOBAL"])  # todo lo demás

#preprocesamiento minimo
cat_features = X.select_dtypes(include=['object']).columns.tolist()

for col in cat_features:
    X[col] = X[col].astype(str).fillna("missing")

#Nans
X = X.fillna(-1)

#división de datos
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.05, 
    random_state=42,
    stratify=y
)

#modelo encontrado en notebooks anteriores
best_params = {
    'random_strength': 2,
    'learning_rate': 0.02,
    'l2_leaf_reg': 1,
    'iterations': 2500,
    'depth': 8,
    'border_count': 64,
    'bagging_temperature': 0.1,
    'auto_class_weights': 'Balanced'
}
#modelo
model = CatBoostClassifier(
    **best_params,
    task_type="GPU",         
    loss_function="MultiClass",
    eval_metric="Accuracy",
    random_seed=42,
    verbose=200
)
#entrenamiento
model.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    use_best_model=True,
    cat_features=cat_features
)
#predicción y rendimiento del modelo
y_pred = model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print("Accuracy:", acc)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Columnas categóricas detectadas:
['E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO', 'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA', 'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD', 'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_EDUCACIONMADRE']
0:	learn: 0.3763216	test: 0.3738954	best: 0.3738954 (0)	total: 154ms	remaining: 6m 25s
200:	learn: 0.4142568	test: 0.4091136	best: 0.4091413 (199)	total: 12.2s	remaining: 2m 19s
400:	learn: 0.4240425	test: 0.4188979	best: 0.4189554 (397)	total: 23.4s	remaining: 2m 2s
600:	learn: 0.4291107	test: 0.4227511	best: 0.4227511 (600)	total: 33.7s	remaining: 1m 46s
800:	learn: 0.4332080	test: 0.4245439	best: 0.4245728 (799)	total: 44.8s	remaining: 1m 35s
1000:	learn: 0.4367196	test: 0.4272969	best: 0.4274080 (962)	total: 56.4s	remaining: 1m 24s
1200:	learn: 0.4394991	test: 0.4279032	best: 0.4287178 (1161)	total: 1m 7s	remaining: 1m 13s
1400:	learn: 0.4419144	test: 0.4287425	best: 0

Otro aspecto a considerar es que en la división de datos para entrenamiento y test, puse muy pocos para test, es decir, entrené el modelo con casi todos los datos. Esto con la finalidad de adquirir un mejor score. Validaba directamente con el archivo de test de kaggle. Esto solo lo intenté en los últimos archivos

In [7]:
from catboost import Pool
#Prueba con datos de test del kaggle
X_test = pd.read_csv(r"C:\Users\USER\Desktop\intro_ML\test.csv", index_col="ID")

#preprocesamiento similar al de entrenamiento
cat_features = model.get_cat_feature_indices()


columnas_modelo = model.feature_names_
X_test = X_test[columnas_modelo]

for col in columnas_modelo:
    if X_test[col].dtype == "object":
        X_test[col] = X_test[col].astype(str).fillna("missing")

test_pool = Pool(X_test, cat_features=cat_features)


y_pred = model.predict(test_pool)

output = pd.DataFrame({
    "ID": X_test.index,
    "RENDIMIENTO_GLOBAL": y_pred[:,0]
})

output.to_csv("submission_catboost.csv", index=False)


In [25]:
y_pred[:,0].shape

(296786,)

A continuación un pipeline que probé con la combinación de varios modelos. No sabía que se podía hacer pero lo llevé a cabo por probar y terminó siendo el que daba mejores resultados. Mezcla el modelo anterior con otros similares, se entrenan cada uno por separado y las predicciones luego se ponderan asignándoles un valor arbitrario de relevancia. Este puede ser optimizado variando estos valores y los hiperparámetros de cada modelo, pero como son nuevos para mí no tengo una intuición sobre ello

In [ ]:
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import lightgbm as lgb

#carga de datos
train = pd.read_csv(r"C:\Users\USER\Desktop\intro_ML\train.csv")
test  = pd.read_csv(r"C:\Users\USER\Desktop\intro_ML\test.csv")

y = train["RENDIMIENTO_GLOBAL"]
X = train.drop(columns=["RENDIMIENTO_GLOBAL"])

test_ids = test["ID"]

#preprocesamiento basico
cat_cols = X.select_dtypes(include=["object"]).columns.tolist()

# Convertir a string y rellenar
for df in [X, test]:
    for col in cat_cols:
        df[col] = df[col].astype(str).fillna("missing")

# Imputación general para números
X = X.fillna(-1)
test = test.fillna(-1)

# Label encode de la objetivo
label_map = {"bajo":0, "medio-bajo":1, "medio-alto":2, "alto":3}
y_encoded = y.map(label_map)

# división train test
X_train, X_val, y_train, y_val = train_test_split(
    X, y_encoded, test_size=0.05, random_state=42, stratify=y_encoded
)

#primer modelo
print("Va en primer modelo")
cat_model = CatBoostClassifier(
    iterations=2000,
    depth=8,
    learning_rate=0.02,
    loss_function="MultiClass",
    eval_metric="Accuracy",
    task_type="GPU",
    auto_class_weights="Balanced",
    verbose=False
)

cat_model.fit(
    X_train, y_train,
    eval_set=(X_val, y_val),
    cat_features=cat_cols
)
print("Ya entrenó el primero")

#Segundo modelo
print("Va en el segundo modelo")

X_train_lgb = X_train.copy()
X_val_lgb   = X_val.copy()
test_lgb    = test.copy()

#preprocesamiento de columnas para este modelo
for col in cat_cols:
    X_train_lgb[col] = X_train_lgb[col].astype("category")
    X_val_lgb[col]   = X_val_lgb[col].astype("category")
    test_lgb[col]    = test_lgb[col].astype("category")

lgb_model = lgb.LGBMClassifier(
    n_estimators=2500,
    learning_rate=0.03,
    max_depth=8,
    num_leaves=60,
    objective="multiclass",
    num_class=4,
    class_weight="balanced",
)

lgb_model.fit(X_train_lgb, y_train)
print("Segundo modelo entrenado")

#Tercer modelo
print("Va en el ultimo modelo")

# Copias solo para XGB
X_train_xgb = X_train.copy()
X_val_xgb   = X_val.copy()
test_xgb    = test.copy()

#prerpocesamiento de columnas para este modelo
for col in cat_cols:
    # Combinar todas las categorías posibles
    all_values = pd.concat([X_train[col], X_val[col], test[col]], axis=0)

    lbl = LabelEncoder()
    lbl.fit(all_values)  

    X_train_xgb[col] = lbl.transform(X_train[col])
    X_val_xgb[col]   = lbl.transform(X_val[col])
    test_xgb[col]    = lbl.transform(test[col])

xgb_model = XGBClassifier(
    n_estimators=2000,
    learning_rate=0.03,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="multi:softprob",
    eval_metric="mlogloss",
    num_class=4,
    tree_method="hist"
)

xgb_model.fit(X_train_xgb, y_train)
print("Modelo listo")

#predicción con el test de kaggle
pred_cat = cat_model.predict_proba(test)
pred_lgb = lgb_model.predict_proba(test_lgb)
pred_xgb = xgb_model.predict_proba(test_xgb)

#mezcla de los modelos
final_proba = 0.45*pred_cat + 0.35*pred_lgb + 0.20*pred_xgb
final_pred = np.argmax(final_proba, axis=1)
#decodificación
decode = {0:"bajo", 1:"medio-bajo", 2:"medio-alto", 3:"alto"}
final_pred_decoded = [decode[p] for p in final_pred]

#generación del csv
submission = pd.DataFrame({
    "ID": test_ids,
    "RENDIMIENTO_GLOBAL": final_pred_decoded
})

submission.to_csv("submission_ensemble.csv", index=False)


Entrenando CatBoost...
CatBoost listo.
Entrenando LightGBM...
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015096 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1893
[LightGBM] [Info] Number of data points in the train set: 657875, number of used features: 20
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes